# The Depo Detection tool
>We finetuned the ESM2 model successfully (92% accuracy)<br>
>The goal now is to stack a RNN layer for a binary classification into Dpo or Not Dpo categories
***
## I. Load prebuilt model 
## II. Stack RNN layer
## III. Train Eval
## IV. Metrics
***

### I. Load the data

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer , AutoTokenizer

import torch 
from torch import nn 
from torch.utils.data import Dataset , DataLoader
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
from Bio import SeqIO
import os 
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 


#model_path = "/home/conchae/PhageDepo_pdb/script_files/models/esm2_t30_150M_UR50D-finetuned-depolymerase/checkpoint-198"

path_work = "/media/concha-eloko/Linux/depolymerase_building"
#path_work = "/home/conchae/PhageDepo_pdb"
# Load the Prebuilt model :
model_path = f"{path_work}/esm2_t12_35M_UR50D-finetuned-depolymerase/checkpoint-198/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# What is the function of that : 
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)



/home/concha-eloko/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /media/concha-eloko/Linux/depolymerase_building/esm2_t12_35M_UR50D-finetuned-depolymerase/checkpoint-198/ were not used when initializing EsmForTokenClassification: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight']
- This IS expected if you are initializing EsmForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenc

In [2]:
# Load the sequences : 
df_depo = pd.read_csv(f"{path_work}/Dpo_domains.phagedepo.0805.final.tsv" , sep = "\t" , header = 0)

df_depo_work =  df_depo[df_depo["Fold"].isin(["right-handed beta-helix", "6-bladed beta-propeller"])]

> Get the negative fasta sequences : <br>
> The interesting annotations : "terminase large subunit" , "helicase" ," DNA polymerase" , "RNaseH" , "methyltransferase"

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/databases/Millard_jan_2023/5Jan2023_vConTACT2_proteins.faa \
/media/concha-eloko/Linux/depolymerase_building  

In [8]:
path_mill_db = f"/{path_work}/5Jan2023_vConTACT2_proteins.faa"
# path_mill_db = "/home/conchae/databases/Millard_jan_2023/5Jan2023_vConTACT2_proteins.faa"

proteins_annot = {record.id : " ".join(record.description.split(" ")[1:]) for record in SeqIO.parse(path_mill_db, "fasta")}
proteins_seq = {record.id : record.seq for record in SeqIO.parse(path_mill_db, "fasta")}


In [28]:
# Make a relevant file for the negative sequences : 
negative_annotation = {"terminase" : [], "helicase" : [], 
                       "DNA polymerase" : [], "RNaseH" : [],
                       "methyltransferase" : [] , "endolysin" : [], 
                       "major head protein" : [], "major tail" :[]}

with open(f"{path_work}/negative_sequences.multi.tsv","w") as outfile :
    for prot_id,description in proteins_annot.items() :
        for neg_annot in negative_annotation :
            if description.lower().count(neg_annot.lower()) > 0 :
                if len(negative_annotation[neg_annot]) < 150 and len(str(proteins_seq[prot_id])) > 200:
                    a = (prot_id , str(proteins_seq[prot_id]))
                    negative_annotation[neg_annot].append(a)
                    break
                else :
                    break
    for neg_annot in negative_annotation :
        for _,tup_neg in enumerate(negative_annotation[neg_annot]) :
            outfile.write(f"{neg_annot}\t{tup_neg[0]}\t{tup_neg[1]}\n")

> Done ! Now make a final DF :

In [3]:
negative_dataset = pd.read_csv(f"{path_work}/negative_sequences.multi.tsv", sep = "\t", names = ["annotation","prot_id","sequence"])
negative_dataset["Label"] = 0 

tmp_neg_df = negative_dataset[["sequence","Label"]]
tmp_neg_df

df_depo_work["Label"] = 1
tmp_pos_df = df_depo_work[["Full_seq", "Label"]]
tmp_pos_df = tmp_pos_df.rename(columns = {"Full_seq" : "sequence"})


/tmp/ipykernel_15752/3990296332.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_depo_work["Label"] = 1


In [8]:
Dataset_df = pd.concat([tmp_pos_df , tmp_neg_df], axis = 0, ignore_index=True)
Dataset_df

,sequence,Label
0,MSSGCGDVLSLADLQTAKKHQIFEAEVITGKSGGVAGGADIDYATN...,1
1,MDQEIKTVIQYPTGSTEFDIPFDYLSRKFVRVSLVSDDNRRLLSNI...,1
2,MADILVTSPYRPFTLPNQFKAVFNGSIYCGTVDAVDPSNSQVQVYK...,1
3,MSSGCGDVLSLNDLQIAKKHQIFEAEVITGKQGGVAGGADIDYATN...,1
4,MTVSIEVDHNDYIGNGVTTSFPYTFRIFKKSDLVVQVADLSENITE...,1
...,...,...
1893,MGVNAANVFIGTPDQSTTGAILSGPVSTSGTVAATIDDVDLSGLTD...,0
1894,MGVNANNVFIGAPDQSTTGAILSGPVLTDPDAPETIDDVDLSGLSD...,0
1895,MSGVFPVVRGIRMRATKVNSCGLPISGPANRIVTDGFVTFNIDPVL...,0
1896,MGVNANNVFIGAPDQSTTGAILSGPVLTDPDAPETIDDVDLSGLSD...,0


>Load the Data : 

In [5]:
class Dpo_Dataset(Dataset):
    def __init__(self, Dataset_df):
        self.sequence = Dataset_df.sequence.values
        self.labels = torch.tensor(Dataset_df["Label"].values, dtype=torch.long)  # Subtract 1 if labels start from 1

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item_domain1 = self.sequence[idx]
        item_domain2 = self.labels[idx]
        return item_domain1, item_domain2

In [6]:
X_train, X_test, y_train, y_test = train_test_split(Dataset_df, Dataset_df["Label"], test_size=0.25, random_state=42)

train_singledata = Dpo_Dataset(X_train)
test_singledata = Dpo_Dataset(X_test)

train_single_loader = DataLoader(train_singledata, batch_size=12, shuffle=True, num_workers=4)
test_single_loader = DataLoader(test_singledata, batch_size=12, shuffle=True, num_workers=4)

In [7]:
train_singledata[0]

('MALISQSIKNLKGGISQQPDILRYPDQGSRQVNGWSSETEGLQKRPPMVFIKTLGDRGALGQAPYIHLINRDENEQYYAVFTGNGIRVFDLAGNEKQVRYPNGSDYIKTSNPRNDLRMVTVADYTFVVNRNVAVQKNTTSVNLPNYNPKRDGLINVRGGQYGRELIVHINGKDVAKYKIPDGSQPAHVNNTDAQWLAEELAKQMRTNLSGWAVNVGQGFIHVAAPSGQQIDSFTTKDGYADQLINPVTHYAQSFSKLPPNAPNGYMVKVVGDASRSADQYYVRYDAERKVWVETLGWNTENQVRWETMPHALVRAADGNFDFKWLEWSPKSCGDIDTNPWPSFVGSSINDVFFFRNRLGFLSGENIILSRTAKYFNFYPASVANLSDDDPIDVAVSTNRISVLKYAVPFSEELLIWSDEAQFVLTASGTLTSKSVELNLTTQFDVQDRARPYGIGRNVYFASPRSSYTSIHRYYAVQDVSSVKNAEDITAHVPNYIPNGVFSICGSGTENFCSVLSHGDPSKIFMYKFLYLNEELRQQSWSHWDFGANVQVLACQSISSDMYVILRNEFNTFLTKISFTKNAIDLQGEPYRAFMDMKIRYTIPSGTYNDDTYNTSIHLPTIYGANFGRGRITVLEPDGKITVFEQPTAGWKSDPWLRLDGNLEGRMVYIGFNIDFVYEFSKFLIKQTADDGSSSTEDIGRLQLRRAWVNYENSGAFDIYVENQSSNWKYSMAGARLGSNTLRAGRLNLGTGQYRFPVVGNAKFNTVSILSDETTPLNIIGCGWEGNYLRRSSGI',
 tensor(1))

***
# The functions and the model architecture 

> LSTM

In [ ]:
class Dpo_classifier(nn.Module):
    def __init__(self, pretrained_model):
        super(Dpo_classifier, self).__init__()
        self.pretrained_model = pretrained_model
        self.rnn = nn.LSTM(input_size=1, hidden_size=524, num_layers=1, batch_first=True, bidirectional=True)
        self.hidden_layer1 = nn.Linear(524*2, 256)  # LSTM is bidirectional now, so output size is hidden_size*2
        self.hidden_layer2 = nn.Linear(256, 128)
        self.hidden_layer3 = nn.Linear(128, 32)  # additional layer
        self.classifier = nn.Linear(32, 2)  # Binary classification
        self.dropout = nn.Dropout(p=0.2)
        self.max_length = 1500

    def make_prediction(self, fasta_txt):
        input_ids = tokenizer.encode(fasta_txt, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.pretrained_model(input_ids)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            labels = self.pretrained_model.config.id2label
            token_probs, token_ids = torch.max(probs, dim=-1)            
            tokens = token_ids.view(1, -1) # ensure 2D shape
            return tokens

    def pad_or_truncate(self, tokens):
        if tokens.size(1) < self.max_length:
            tokens = F.pad(tokens, (0, self.max_length - tokens.size(1)))
        elif tokens.size(1) > self.max_length:
            tokens = tokens[:, :self.max_length]
        return tokens

    def forward(self, sequences):
        batch_size = len(sequences)
        tokens_batch = []
        for seq in sequences:
            tokens = self.make_prediction(seq)
            tokens = self.pad_or_truncate(tokens)
            tokens_batch.append(tokens)
        
        outputs = torch.cat(tokens_batch).view(batch_size, self.max_length, -1)
        outputs = outputs.float()
        
        _, (hidden_state, _) = self.rnn(outputs)
        out = hidden_state.view(hidden_state.size(1), -1)  # Concatenate the hidden states from both directions
        out = F.relu(self.hidden_layer1(out))
        out = self.dropout(out)
        out = F.relu(self.hidden_layer2(out))
        out = self.dropout(out)
        out = F.relu(self.hidden_layer3(out))  # Pass through the additional layer
        out = self.classifier(out)
        return out , outputs

<div class="alert alert-block alert-warning">
LSTM does not really work (accuracy 0.62)

> Conv (2 conv layers)

In [ ]:
class Dpo_classifier(nn.Module):
    def __init__(self, pretrained_model):
        super(Dpo_classifier, self).__init__()
        self.max_length = 1024
        self.pretrained_model = pretrained_model
        self.conv1 = nn.Conv1d(1, 64, kernel_size=5, stride=1)  # Convolutional layer
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=1)  # Convolutional layer
        self.fc1 = nn.Linear(128 * (self.max_length - 2 * (5 - 1)), 32)  # calculate the output shape after 2 conv layers
        self.classifier = nn.Linear(32, 2)  # Binary classification

    def make_prediction(self, fasta_txt):
        input_ids = tokenizer.encode(fasta_txt, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.pretrained_model(input_ids)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            token_probs, token_ids = torch.max(probs, dim=-1)            
            tokens = token_ids.view(1, -1) # ensure 2D shape
            return tokens

    def pad_or_truncate(self, tokens):
        if tokens.size(1) < self.max_length:
            tokens = F.pad(tokens, (0, self.max_length - tokens.size(1)))
        elif tokens.size(1) > self.max_length:
            tokens = tokens[:, :self.max_length]
        return tokens

    def forward(self, sequences):
        batch_size = len(sequences)
        tokens_batch = []
        for seq in sequences:
            tokens = self.make_prediction(seq)
            tokens = self.pad_or_truncate(tokens)
            tokens_batch.append(tokens)
        
        outputs = torch.cat(tokens_batch).view(batch_size, 1, self.max_length)  # ensure 3D shape
        outputs = outputs.float()  # Convert to float
        
        out = F.relu(self.conv1(outputs))
        out = F.relu(self.conv2(out))
        out = out.view(batch_size, -1)  # Flatten the tensor
        out = F.relu(self.fc1(out))
        out = self.classifier(out)
        return out, outputs

<div class="alert alert-block alert-success">
The Convolutional layers works well (0.98 accuracy)
</div>

Let's try with a single conv layer

> Conv (1 conv layer)

In [ ]:
class Dpo_classifier(nn.Module):
    def __init__(self, pretrained_model):
        super(Dpo_classifier, self).__init__()
        self.max_length = 1024
        self.pretrained_model = pretrained_model
        self.conv1 = nn.Conv1d(1, 64, kernel_size=5, stride=1)  # Convolutional layer
        self.fc1 = nn.Linear(64 * (self.max_length - (5 - 1)), 32)  # calculate the output shape after 1 conv layer
        self.classifier = nn.Linear(32, 2)  # Binary classification

    def make_prediction(self, fasta_txt):
        input_ids = tokenizer.encode(fasta_txt, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.pretrained_model(input_ids)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            token_probs, token_ids = torch.max(probs, dim=-1)            
            tokens = token_ids.view(1, -1) # ensure 2D shape
            return tokens

    def pad_or_truncate(self, tokens):
        if tokens.size(1) < self.max_length:
            tokens = F.pad(tokens, (0, self.max_length - tokens.size(1)))
        elif tokens.size(1) > self.max_length:
            tokens = tokens[:, :self.max_length]
        return tokens

    def forward(self, sequences):
        batch_size = len(sequences)
        tokens_batch = []
        for seq in sequences:
            tokens = self.make_prediction(seq)
            tokens = self.pad_or_truncate(tokens)
            tokens_batch.append(tokens)
        
        outputs = torch.cat(tokens_batch).view(batch_size, 1, self.max_length)  # ensure 3D shape
        outputs = outputs.float()  # Convert to float
        
        out = F.relu(self.conv1(outputs))
        out = out.view(batch_size, -1)  # Flatten the tensor
        out = F.relu(self.fc1(out))
        out = self.classifier(out)
        return out, outputs

***
# Training the model and Evaluation : 

In [35]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
# Initialize model
model_classifier = Dpo_classifier(model)
model_classifier.train()

optimizer = optim.Adam(model_classifier.parameters(), lr=0.001)  # Set learning rate
criterion = nn.CrossEntropyLoss()  # Set loss function

epochs = 100  # Number of training epochs

# Training loop
for epoch in range(epochs):
    # Training
    model_classifier.train()
    epoch_loss = 0
    epoch_correct = 0
    total_samples = 0
    for i, (sequences, labels) in enumerate(train_single_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        outputs, _ = model_classifier(sequences)
        loss = criterion(outputs, labels)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        epoch_correct += (predicted == labels).sum().item()
        # Accumulate loss
        epoch_loss += loss.item()
    print(f'Epoch {epoch + 1}, Training Loss: {epoch_loss / len(train_single_loader):.4f}, Training Accuracy: {epoch_correct / total_samples:.4f}')
    # Evaluation
    model_classifier.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for sequences, labels in test_single_loader:
            outputs, _ = model_classifier(sequences)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.numpy())            
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')  
    recall = recall_score(y_true, y_pred, average='binary')  
    f1 = f1_score(y_true, y_pred, average='binary')  
    print(f'Testing Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
    
print('Finished Training')

torch.save(model_classifier.state_dict(), f"{path_work}/DepoDetection.model")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [11]:
#!/bin/bash
#BATCH --job-name=final_model__
#SBATCH --qos=short
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=5 
#SBATCH --mem=50gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=final_model__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate embeddings

python /home/conchae/PhageDepo_pdb/script_files/final_model.py

0 ('MSSGCGDVLSLADLQTAKKHQIFEAEVITGKSGGVATGADIDYATNQVTGQTQKTLPAVLRDAGFSPVSWDFSTGGTLTVNDRDKVVYDPVSETWYSYAGTLPVVVPASFNPVGNANWKPQTDPNLRNDLASSTAGLGASLVSFSNGNTVETLSDADGAKNIGSGERSLLARNNDIKHSGDFSTLQAAVDASLPKNDLLISPGEYTEKVTIGSAQLKGVGGAVVLKTPADFTNTVQVNLSTPHWQFRHSGGFAIDGSGTTGAVGISFDPSDQYSGRHNFSDLYIHNINKAIQKPSGNIGNTWRNIGVSTCDWGYYAISGSEMHCGADTLYNIHFDGISTYAVYLNGTADNGGIGGWWLKDSIIEASGGGGIYLKSKSGDCPTSPCGVSNVWTEAIATSSAVQVDGVAQKPRVLKLVDTAIFFAEYSYLNNIELSNSNLVTYGCRFDNADGNQDIVVDAQSTIVAHDVYLNGSSGKDVIVESVASQSATVATTNLSLRGNLTRGRVFNTPTGNKLKAITFDSGSHNFSGSGTVNGSTVSDGLHAATCTEFSFPGSGLYEMVASRTTLTSGRWYVWGVNSRLQSGTADVSITSGITMGSVYTKSGEWISTFGVGKASANGTVSLYVSTAGGSGAVIRFSDFFIAEFTTQAQALAFANSRMSLA', 'MGYNVIRMIRRGLPQVGVAPYGQVHAHSTGNPNSTAKNEADYMQHKDINTGFYTHVVGNGQVYQVAEVNRGAWDVGGGWNQWGYASVELIESHRNKDEFMRDYRIYCELLYDLAKQAGLPTTVDVGNTGIITHNYATHNQPNNGSDHVDPIPYLAKWGINLEQFRKDVANAKGNNSGKPAPAPAQPKPSTPKGHDEAVAASKPKHQGNAWGKLDKYREEPAGKIRVAGWLVPDKPNGAIGSTAWVLFMQHGTNKELTRIQSQGIKRPDVKKQYGYRGGDALGFDVTVDKKQFKGKKVDVILRRANNGKNGESPVNDVRIDDIYLTL', 'M